# Singleton

The Singleton pattern ensures that no more than one instance of a class can be created.
What's more, the Singleton allows to access this instance from anywhere in the app.

To allow that, we make a trick: we define the constructor of the class as a private method, which means it is not accesible from outside that class; At the same time, the class has a static method that checks if some instance of the class exists and otherwise it calls the constructor. This method is accessible from anywhere. By doing that, we leverage all the responsability of creating instances to the class itself.

There are many ways of implementing a Singleton in Python. Here we are using metaclasses:

In [5]:
class SingletonMeta(type):

    _instances = {}  # The created instances of each singleton class

    def __call__(cls, *args, **kwargs):

        if cls not in cls._instances:
            # If no instance exists, we create one and add
            # it to the dictionary of created instances
            instance = super().__call__(*args, **kwargs)
            cls._instances[cls] = instance
        return cls._instances[cls]


class Singleton(metaclass=SingletonMeta):
    # A Singleton class

    def __init__(self, value: str) -> None:
        # We will use this variable to check if our singleton pattern
        # works
        self.value = value

    def do_something(self) -> None:
        print("Do something...")


if __name__ == "__main__":

    # Let's create two instances of Singleton
    s1 = Singleton("s1")
    s2 = Singleton("s2")

    # Actually, both variables contain the same instance
    print(id(s1) == id(s2))
    print(s1.value)
    print(s2.value)


True
s1
s1


Singleton is a discused pattern: some people even consider it an antipattern.

Some *possible* use cases for a Singleton are:
- Logging
- Reading configuration files at startup time and encapsulate them in a unique instance which can be accessed globally.
- Database connections
- Hardware interface access
- Cache

In general, Singleton can be used to manage resources that should be accessed only individually.

However:
- Singleton might complicate unit testing. 
- It must be modified when working with multithreading to avoid that the threads create a Singleton multiple times.

When using multithreading and Singleton, we must use a lock before checking and creating the Singleton instance. Otherwise, multiple threads could access the \_\_call\_\_ method at the same time, pass the conditional and create multiple Singletons simultaneously. The modified code would be:

In [27]:
# import the threading modules
from threading import Lock, Thread


class SingletonMeta(type):

    _instances = {}  # The created instances of each singleton class
    _lock = Lock()  # The lock to access the call method.

    def __call__(cls, *args, **kwargs):

        with cls._lock:
            # If a thread is inside this point, the other will wait before
            # lock 'till it's finished
            if cls not in cls._instances:
                # If no instance exists, we create one and add
                # it to the dictionary of created instances
                instance = super().__call__(*args, **kwargs)
                cls._instances[cls] = instance
            return cls._instances[cls]


class Singleton(metaclass=SingletonMeta):
    # A Singleton class

    def __init__(self, value: str) -> None:
        # We will use this variable to check if our modified singleton
        # pattern works
        self.value = value

    def do_something(self) -> None:
        print("Do something...")


def test_singleton(value):
    # A function to test our singleton
    s = Singleton(value)
    print(s.value)


if __name__ == "__main__":

    # Let's create two threads. Each of them try to instanciate a Singleton,
    # but only the first one (the first who adquires the lock) is able to give
    # a value to it
    thread1 = Thread(target=test_singleton, args=("Thread 1 wins!",))
    thread2 = Thread(target=test_singleton, args=("Thread 2 wins!",))

    thread1.start()
    thread2.start()


Thread 1 wins!
Thread 1 wins!
